In [ ]:
from pprint import pprint
from girder_client import GirderClient
from dive_utils import fromMeta

GIRDER_API_KEY="maLOMfOOhqzgWawfkZwblEvDWdXTcGZTu31gp9P4"
diveclient = GirderClient(apiUrl='https://viame.kitware.com/api/v1')
token = diveclient.authenticate(apiKey=GIRDER_API_KEY)
print(f'Authenticated with key={token} \n\tand token={diveclient.token}')

In [ ]:
# Load dataset metadata
DATASET_ID="5e4ebcaa78ed364cd0f5e3b9"

dataset_info = diveclient.getFolder(DATASET_ID)
dataset_name = dataset_info['name']
dataset_type = fromMeta(dataset_info, 'type')
dataset_fps  = fromMeta(dataset_info, 'fps')

print(f'Loaded dataset: {dataset_name}\n')
pprint(dataset_info)

# Load dataset source media

In [ ]:
if dataset_type == 'image-sequence':
    dataset_media = diveclient.get(
        f'viame/valid_images', parameters={'folderId': DATASET_ID})
    dataset_media = [item['name'] for item in dataset_media]
else:
    dataset_media = diveclient.get(
        f'viame_detection/clip_meta',
        parameters={'folderId': DATASET_ID}
    )

pprint(dataset_media)

# Load annotations and summarize contents

In [ ]:
from dive_tasks import summary

dataset_annotations_json = diveclient.get(
    'viame_detection',
    parameters={'folderId': DATASET_ID},
)

all_summary = {}
summary.summarize_annotations(DATASET_ID, dataset_annotations_json, all_summary)
pprint(f'Summary of {DATASET_ID}')
pprint([ s.dict() for s in all_summary.values()])

# Load detections as CSV

In [56]:
dataset_annotations_csv = diveclient.get(f'viame_detection/{DATASET_ID}/export_detections', jsonResp=False)
print(dataset_annotations_csv.text)

('# 1: Detection or Track-id,2: Video or Image Identifier,3: Unique Frame '
 'Identifier,4-7: Img-bbox(TL_x,TL_y,BR_x,BR_y),8: Detection or Length '
 'Confidence,9: Target Length (0 or -1 if invalid),10-11+: Repeated '
 'Species,Confidence Pairs or Attributes\r\n'
 '# metadata -\r\n'
 '# Written on Thu Apr  8 18:32:50 2021 by: dive:python\r\n'
 '0,SC3-Camera4_03-16-18_13-01-40.000.avi_000001.png,0,1018,441,1273,633,1.0,-1,LUTJANUSCAMPECHANUS-170151107,1.0\r\n'
 '0,SC3-Camera4_03-16-18_13-01-40.000.avi_000002.png,1,1007,429,1263,622,1.0,-1,LUTJANUSCAMPECHANUS-170151107,1.0\r\n'
 '0,SC3-Camera4_03-16-18_13-01-40.000.avi_000003.png,2,1044,383,1300,576,1.0,-1,LUTJANUSCAMPECHANUS-170151107,1.0\r\n'
 '0,SC3-Camera4_03-16-18_13-01-40.000.avi_000004.png,3,1011,368,1408,524,1.0,-1,LUTJANUSCAMPECHANUS-170151107,1.0\r\n'
 '0,SC3-Camera4_03-16-18_13-01-40.000.avi_000005.png,4,1144,311,1494,511,1.0,-1,LUTJANUSCAMPECHANUS-170151107,1.0\r\n'
 '0,SC3-Camera4_03-16-18_13-01-40.000.avi_000006.png,5,1226